# FastAPI

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/tools/fastapi.ipynb)

FastAPI est une library de code Python pour construire des API json RestFul, notamment pour déployer des modèles d'intelligence artificielle dans les architectures cloud.

- https://github.com/tiangolo/fastapi

On préfère FastAPI aux alternatives suivantes :

- https://github.com/django/django
- https://github.com/pallets/flask/
- https://github.com/tornadoweb/tornado

L'exemple qui suit implémente l'accès à un modèle de régression linéaire par une API json RESTful.

In [26]:
import os
import sys

# Supprimer les commentaires pour installer (requirements.txt)
# !pip3 install -q -U scikit-learn
# !pip3 install -q -U pydantic

# À installer dans tous les cas pour Google Colab et Github
if ('google.colab' in sys.modules) or ('CI' in os.environ):
    !pip3 install -q -U "fastapi[all]"
    # !pip3 install -q -U "uvicorn[standard]"

In [27]:
# Les variables python sont accessibles depuis les commandes shell
work_directory = './fastapi'
app_directory = work_directory + '/app'

!mkdir -p $app_directory

# Supprimer le commentaire pour créer le fichier de requirements
# !pip3 freeze > $work_directory/requirements.txt

## Création et sérialisation d'un modèle SciKit-Learn

In [28]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression

# Création de données synthétiques
X, y = make_regression(n_samples=100, n_features=1, random_state=123)

# Entrainement d'un modèle de régression linéaire avec scikit-learn
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [29]:
import pickle
pickle.dump(model, open(f'{app_directory}/model.pkl','wb'))

# alternativement
# import joblib
# joblib.dump(model, f'{app_directory}/model.joblib')

## Codage de l'application FastAPI

In [30]:
#%%writefile $app_directory/__init__.py
# Application FastAPI (writefile a besoin de contenu pour écrire)

In [31]:
#%%writefile $app_directory/main.py
import pickle
from contextlib import asynccontextmanager
from fastapi import FastAPI
from pydantic import BaseModel

# app_directory = '.' # enlever le commentaire dans main.py
models = {} # au cas où l'API nécessite plusieurs modèles 

@asynccontextmanager
async def lifespan(app: FastAPI):
    # Ne pas oublier de livrer le(s) fichier(s) de modèle(s) avec l'application FastAPI
    models['linear'] = pickle.load(open(f'{app_directory}/model.pkl', 'rb'))
    print('Linear model loaded')
    # alternativement
    # models['linear'] = joblib.load(f'{app_directory}/model.joblib')
    yield
    # libérer les ressources si nécessaire

app = FastAPI(title="Application FastAPI", lifespan=lifespan)

# Définition de la classe Item pour le schéma json de la requête POST /predict
class Item(BaseModel):
    x: float
    # x2: float
    # x3: float

# requête POST /predict pour prédire la valeur de y à partir de la valeur de x
# en utilisant le modèle linéaire chargé dans le dictionnaire models    
@app.post("/predict")
async def predict(item:Item):
    y_pred = models['linear'].predict([[item.x]])
    return {'y_pred': y_pred[0]}

Voir aussi [Pydantic](./pydantic.ipynb).

## Lancement de l'application FastAPI

Dans un bloc-note Jupyter, l'application FastAPI codée ci-dessus peut être lancée par le code ci-dessous, en notant que le serveur reste actif tant que le code de la cellule n'est pas manuellement interrompu:

In [32]:
# Ne pas exécuter dans Google Colab ou dans les tests
#if ('google.colab' not in sys.modules) or sys.argv[0].endswith('pytest'):
#    import uvicorn
#    config = uvicorn.Config(app)
#    server = uvicorn.Server(config)
#    await server.serve()

Vous pouvez tester manuellement l'API tant que le serveur reste actif (jusqu'à ce que vous stoppiez l'exécution de la cellule ci-dessus):

- soit en ouvrant un navigateur à l'adresse http://127.0.0.1:8000/docs,
- soit en exécutant la commande suivante dans un terminal:

```shell
curl -X POST http://127.0.0.1:8000/predict -H 'Content-Type: application/json' -d '{"x":1.0}'
```

Dans un environnement de production, il est recommandé de "containeriser" l'application FastAPI avec [Docker](https://fastapi.tiangolo.com/deployment/docker). Pour cela, utilisez le fichier Dockerfile produit par la cellule ci-dessous , en supposant l'organisation de fichiers suivante, dans laquelle `__init__.py` est un fichier vide:

```cmd
.
├── app
│   ├── __init__.py
│   ├── main.py
│   └── model.pkl
├── Dockerfile
└── requirements.txt
```

Alternativement vous pouvez aussi considérer les architectures sans serveur (serverless) fournies par [Amazon Web Services](./aws.ipynb), [Google Cloud](./google.ipynb) et [Microsoft Azure](./azure.ipynb)

In [33]:
%%writefile $work_directory/Dockerfile
FROM python:3.12-slim

# Mise à jour de système linux (à considérer)
RUN apt -y update

# Répertoire de travail dans l'environnement Docker
WORKDIR /code

# Fichier requirements.txt obtenu par pip freeze > requirements.txt
COPY ./requirements.txt /code/requirements.txt

# Installation des dépendances depuis le fichier requirements.txt
RUN pip install --no-cache-dir --upgrade -r /code/requirements.txt

# Copie du code source dans le répertoire de travail, notamment
# - l'application fastAPI ./app/main.py 
COPY ./app /code/app

# Expose le port 80 pour l'application (préférer un port non attribué et l'usage d'un proxy comme nginx)
EXPOSE 80

# Exécution de l'application avec uvicorn
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "80", "--reload"]

Overwriting ./fastapi/Dockerfile


## Test de l'application FastAPI

> <span style='color:red'>Pensez à stopper manuellement l'exécution de la cellule contenant la ligne `await server.serve()` pour exécuter le test ci-dessous.</span>

In [34]:
from fastapi.testclient import TestClient

client = TestClient(app)

response = client.post("/predict", json={"x": 1.0})
assert response.status_code == 200
y_pred = response.json()['y_pred']
assert type(y_pred) == float
print('y_pred:', y_pred)

KeyError: 'linear'

In [ ]:
# Ménage
!rm -rf $work_directory